In [ ]:
import streamlit as st
import os
from typing import List
import json
import pandas as pd
from datetime import datetime

# --- Clases simuladas para evaluación real ---
class Structure:
    def __init__(self, label, volume):
        self.label = label
        self.volume = volume
        self.dose_axis = list(range(0, 10001, 100))  # Dummy axis
        self.cumulated_percent_volume_axis = [100 - (i / 100) for i in self.dose_axis]  # Simple decay

    def volume_function(self, dose):
        return max(0.0, 100 - dose / 100)

    def dose_function(self, volume):
        return max(0.0, (100 - volume) * 100)

    @property
    def mean(self):
        return sum(self.dose_axis) / len(self.dose_axis)

class Constraint:
    def __init__(self, constraints_chart_line):
        self.structure_name, self.type, self.ideal_dose, self.ideal_volume, self.acceptable_dose, self.acceptable_volume = constraints_chart_line
        self.structure_name = self.structure_name.upper()
        self.VERIFIED_IDEAL = (False, 0.0)
        self.VERIFIED_ACCEPTABLE = (False, 0.0)
        self.ACCEPTABLE_LV_AVAILABLE = self.acceptable_dose != 'None'

    def _evaluate(self, structure, ref1, ref2):
        constraint_types = ['V(D)>V_%', 'V(D)>V_cc', 'V(D)<V_%', 'V(D)<V_cc', 'D(V_%)<D', 'D(V_cc)<D', 'Dmax', 'Dmedia']

        if self.type in constraint_types[:4]:
            is_superior = self.type in (constraint_types[2], constraint_types[3])
            abs_volume = self.type in (constraint_types[1], constraint_types[3])

            ref_dose = float(ref1)
            ref_vol = float(ref2)
            result = structure.volume_function(ref_dose)
            result = result * structure.volume / 100.0 if abs_volume else result
            PASS = result <= ref_vol if is_superior else result >= ref_vol
            return (PASS, round(result, 1))

        elif self.type in constraint_types[4:6]:
            abs_volume = self.type == constraint_types[5]
            ref_vol = float(ref1)
            ref_vol = ref_vol * 100.0 / structure.volume if abs_volume else ref_vol
            ref_dose = float(ref2)
            result = structure.dose_function(ref_vol)
            PASS = result <= ref_dose
            return (PASS, round(result, 1))

        elif self.type in constraint_types[6:]:
            dmax = self.type == constraint_types[6]
            dmed = self.type == constraint_types[7]
            ref_dose = float(ref1)
            result = structure.mean if dmed else structure.dose_function(2.0)
            PASS = result <= ref_dose
            return (PASS, round(result, 1))

        else:
            return (False, 'None')

    def verify(self, structure):
        self.VERIFIED_IDEAL = self._evaluate(structure, self.ideal_dose, self.ideal_volume)
        if not self.VERIFIED_IDEAL[0] and self.ACCEPTABLE_LV_AVAILABLE:
            self.VERIFIED_ACCEPTABLE = self._evaluate(structure, self.acceptable_dose, self.acceptable_volume)

# --- Dummy loading logic ---
def load_dvh_from_file(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def get_recent_dvhs(directory: str, extension: str = ".json") -> List[str]:
    files = [f for f in os.listdir(directory) if f.endswith(extension)]
    return sorted(files, key=lambda x: os.path.getmtime(os.path.join(directory, x)), reverse=True)[:10]

# --- Streamlit UI ---
st.set_page_config(layout="wide")
st.title("DVH Constraint Assessment")

st.sidebar.header("Carga de Archivo")
dvh_directory = st.sidebar.text_input("Directorio de DVHs", value="./dvh_files")

if os.path.exists(dvh_directory):
    recent_files = get_recent_dvhs(dvh_directory)
    selected_file = st.sidebar.selectbox("Historial de archivos recientes", recent_files)
    full_path = os.path.join(dvh_directory, selected_file)
    dvh_data = load_dvh_from_file(full_path)
else:
    st.warning("Directorio no válido")
    st.stop()

col1, col2 = st.columns([1, 2])

st.header("Asignación de nombres y volumenes")
renaming_dict = {}
volume_dict = {}

for structure in dvh_data.get("structures", []):
    name = structure["label"]
    with col2:
        new_name = st.text_input(f"Renombrar '{name}'", value=name, key=f"name_{name}")
        volume = st.text_input(f"Volumen para '{new_name}' (cc)", key=f"vol_{new_name}")
    renaming_dict[name] = new_name
    try:
        volume_dict[new_name] = float(volume)
    except ValueError:
        volume_dict[new_name] = None

st.subheader("Resultados de constraints")

if st.button("Verificar constraints"):
    constraints = [
        Constraint(["PTV", "D(V_cc)<D", 95, 0.03, 90, 0.03]),
        Constraint(["BLADDER", "V(D)<V_%", 65, 50, 70, 60]),
        Constraint(["RECTUM", "Dmax", 75, None, 80, None]),
    ]

    results = []
    export_data = []
    for constraint in constraints:
        name = constraint.structure_name
        matched = next((v for k, v in renaming_dict.items() if v.upper() == name), None)
        if matched and matched in volume_dict and volume_dict[matched]:
            struct = Structure(matched, volume_dict[matched])
            constraint.verify(struct)
            if constraint.VERIFIED_IDEAL[0]:
                result = f"✅ {matched}: Cumple ideal ({constraint.VERIFIED_IDEAL[1]})"
            elif constraint.VERIFIED_ACCEPTABLE[0]:
                result = f"⚠️ {matched}: Cumple aceptable ({constraint.VERIFIED_ACCEPTABLE[1]})"
            else:
                result = f"❌ {matched}: No cumple"
        else:
            result = f"❗ {name}: No hay volumen asignado"
        results.append(result)
        export_data.append({
            "Structure": name,
            "Renamed": matched,
            "Volume": volume_dict.get(matched),
            "Result": result
        })

    for res in results:
        st.text(res)

    df = pd.DataFrame(export_data)
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"assessment_results_{now}.xlsx"
    df.to_excel(output_filename, index=False)
    st.success(f"Resultados exportados a {output_filename}")
